In [7]:
# Install
!pip install transformers torch scipy

# Generate music
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import scipy.io.wavfile
import torch

# Load model
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Generate samples
prompts = [
    "upbeat electronic dance music with heavy bass",
    "calm piano melody with strings",
    "rock music with electric guitar solo",
    "jazz saxophone with drums",
    "classical orchestral piece"
]

for idx, prompt in enumerate(prompts):
    inputs = processor(text=[prompt], padding=True, return_tensors="pt").to(device)

    # Generate audio (256 tokens = ~5 seconds, 512 = ~10 seconds)
    audio_values = model.generate(**inputs, max_new_tokens=512)

    # Save as WAV
    sampling_rate = model.config.audio_encoder.sampling_rate
    scipy.io.wavfile.write(
        f"musicgen_sample_{idx}.wav",
        rate=sampling_rate,
        data=audio_values[0, 0].cpu().numpy()
    )
    print(f"✓ Generated sample {idx}: {prompt}")

print("\nDone! Files ready to download.")

✓ Generated sample 0: upbeat electronic dance music with heavy bass
✓ Generated sample 1: calm piano melody with strings
✓ Generated sample 2: rock music with electric guitar solo
✓ Generated sample 3: jazz saxophone with drums
✓ Generated sample 4: classical orchestral piece

Done! Files ready to download.


In [8]:
!pip install transformers torch scipy tqdm faker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.1 MB/s eta 0:00:00


In [11]:
import os
import json
import random
import torch
import scipy.io.wavfile
from tqdm import tqdm
from faker import Faker
from transformers import AutoProcessor, MusicgenForConditionalGeneration

# -----------------------
# CONFIG
# -----------------------
OUTPUT_DIR = "musicgen_10k_dataset"
os.makedirs(OUTPUT_DIR, exist_ok=True)

NUM_SAMPLES = 2000       # total number of clips to generate
BATCH_SIZE = 4            # more = faster, but uses more VRAM
CLIP_TOKENS = 256         # ~5 seconds
SAVE_META_EVERY = 100     # safety checkpoint
RESUME = True             # resumes if directory already has files
# -----------------------

fake = Faker()

# Prompt pools
genres = [
    "electronic", "house", "lofi", "jazz", "classical", "rock", "metal",
    "piano", "ambient", "trap", "hip hop", "folk", "instrumental", "acoustic"
]

moods = [
    "dark", "happy", "melancholic", "energetic", "calm",
    "dreamy", "epic", "mysterious", "uplifting", "aggressive"
]

instruments = [
    "piano", "strings", "synths", "violin", "electric guitar",
    "drums", "saxophone", "flute", "bass guitar", "harp"
]

# Load model + processor once
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small").to(device)

# Metadata store
metadata_path = os.path.join(OUTPUT_DIR, "metadata.json")
metadata = []

# Resume logic
start_idx = 0
if RESUME:
    existing = [f for f in os.listdir(OUTPUT_DIR) if f.endswith(".wav")]
    start_idx = len(existing)
    print(f"Resuming: {start_idx} samples already present.")

# -----------------------
# Generate N samples
# -----------------------

for i in tqdm(range(start_idx, NUM_SAMPLES, BATCH_SIZE)):

    batch_prompts = []
    for _ in range(BATCH_SIZE):
        genre = random.choice(genres)
        mood = random.choice(moods)
        inst = random.choice(instruments)

        # synthetic natural language prompt
        prompt = f"{mood} {genre} music featuring {inst}"
        batch_prompts.append(prompt)

    # Tokenize
    inputs = processor(text=batch_prompts, padding=True, return_tensors="pt").to(device)

    # Generate audio
    audio_batch = model.generate(**inputs, max_new_tokens=CLIP_TOKENS)

    # Save each file
    for j in range(len(batch_prompts)):
        idx = i + j
        if idx >= NUM_SAMPLES:
            break

        filename = f"sample_{idx}.wav"
        filepath = os.path.join(OUTPUT_DIR, filename)

        # write WAV
        sampling_rate = model.config.audio_encoder.sampling_rate
        scipy.io.wavfile.write(
            filepath,
            rate=sampling_rate,
            data=audio_batch[j, 0].cpu().numpy()
        )

        # record metadata
        metadata.append({
            "id": idx,
            "prompt": batch_prompts[j],
            "file": filename
        })

    # save metadata periodically
    if idx % SAVE_META_EVERY == 0:
        with open(metadata_path, "w") as f:
            json.dump(metadata, f, indent=2)

    # memory cleanup
    del audio_batch
    torch.cuda.empty_cache()

# final save
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=2)

print("🎉 Done! All samples generated.")


Resuming: 440 samples already present.


100%|██████████| 390/390 [45:10<00:00,  6.95s/it]

🎉 Done! All samples generated.


In [16]:
!zip -r /content/musicgen_10k_dataset.zip /content/musicgen_10k_dataset


  adding: content/musicgen_10k_dataset/ (stored 0%)
  adding: content/musicgen_10k_dataset/sample_42.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1766.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1387.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1260.wav (deflated 8%)
  adding: content/musicgen_10k_dataset/sample_1751.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_533.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1501.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1850.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1085.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1509.wav (deflated 8%)
  adding: content/musicgen_10k_dataset/sample_1589.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1240.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1205.wav (deflated 7%)
  adding: content/musicgen_10k_dataset/sample_1794.wav

In [17]:
from google.colab import files
files.download("/content/musicgen_10k_dataset.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>